Q1

In [1]:
import json

with open('documents.json', 'rt') as f_in:
    documents = json.load(f_in)

len(documents)

948

Q 2,3

Q2 answer can be found at - https://sbert.net/docs/sentence_transformer/pretrained_models.html

In [2]:
from tqdm.auto import tqdm

In [3]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"

from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-MiniLM-L12-v2")

In [4]:
len(model.encode("This is a simple sentence"))

384

In [5]:
#created the dense vector using the pre-trained model
operations = []
for doc in tqdm(documents):
    # Transforming the title into an embedding using the model
    doc["question_vector"] = model.encode(doc["question"]).tolist()
    operations.append(doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [6]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '97d47c998a64', 'cluster_name': 'docker-cluster', 'cluster_uuid': '3_hR6bhbRfSFv8fBYSTcbg', 'version': {'number': '8.7.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'f229ed3f893a515d590d0f39b05f68913e2d9b53', 'build_date': '2023-04-27T04:33:42.127815583Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

Q 4,5

In [13]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(
    index=index_name,
    settings={
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    mappings={
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_vector":{
                "type":"dense_vector", "dims": 384,
                "index":True, "similarity": "cosine"
            },
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [14]:
for doc in tqdm(operations):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/948 [00:00<?, ?it/s]

In [15]:
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

knn_query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, knn=knn_query,
                       source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'TGFcj5ABp2H1AM4wJmUK',
  '_score': 0.85204804,
  '_source': {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
   'section': 'General course-related questions',
   'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'bWFcj5ABp2H1AM4wJmXT',
  '_score': 0.8381672,
  '_source': {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
   'section': 'General course-related questions',
   'question': 'How can we contribute to the course?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'D2Fcj5ABp2H1AM4wL2eR',
  '_score': 0.8284228,
  '_source': {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read

Q 6

In [16]:
knn_query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {
            "section": "General course-related questions"
        },
    },
    knn=knn_query,
    size=5,
    source=["text", "section", "question", "course"]
)

response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'TGFcj5ABp2H1AM4wJmUK',
  '_score': 3.4953294,
  '_source': {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
   'section': 'General course-related questions',
   'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'bWFcj5ABp2H1AM4wJmXT',
  '_score': 3.4814484,
  '_source': {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
   'section': 'General course-related questions',
   'question': 'How can we contribute to the course?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'D2Fcj5ABp2H1AM4wL2eR',
  '_score': 3.471704,
  '_source': {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read e